In [ ]:
def get_panel_df(sheet):
    # load sheet
    df = pd.read_excel("/Users/martinszyska/Dropbox/Icke/Work/Topics/CONAN/LungCancerPanels.xlsx", sheet_name=sheet)
    # extract Gene + ID
    df.loc[:, ['Gene', 'ID']] = df['Gen Transkript'].str.strip(" ").str.extract(r"(?P<Gene>[A-Z]+[0-9]*) +(?P<ID>NM_[0-9]+)")
    df.loc[:, "Exon"] = df['Genbereiche'].str.extract(r"Exon ?(?P<Exon>[^A-Za-z]+)", expand=False).str.replace("[ +]", "", regex=True)
    df.loc[:, "Intron"] = df['Genbereiche'].str.extract(r"Intron ?(?P<Intron>[^A-Za-z]+)", expand=False).str.replace("[ +]", "", regex=True).fillna("")
    return df.loc[:, ["Gene", "ID", "Exon", "Intron"]]

In [ ]:
nngn = get_panel_df("nNGM")
nngn

In [ ]:
def expand_range(string):
    splt = string.split('-')
    if len(splt) == 1:
        return splt
    
    return list(range(int(splt[0]), int(splt[1]) + 1))


def get_rows(row):
    dfs = []
    for t in ["Exon", 'Intron']:
        ll = [expand_range(n) for n in row[t].split(",")]
        l = [f"{t}{x}" for i in ll for x in i]
        if l != ["Intron"]:
            df = pd.DataFrame({'Exon':l})
            dfs.append(df)
    df = pd.concat(dfs)
    # add data
    for col in ["Gene", "ID"]:
        df[col] = row[col]    
    return df.loc[:, ["Gene", "ID", "Exon"]]


def expand_df(df):
    dfs = []
    for i, row in df.iterrows():
        dfs.append(get_rows(row))
    return pd.concat(dfs)

In [ ]:
nngn_df = expand_df(nngn)
nngn_df

In [ ]:
oncomine = get_panel_df("Oncomine Focus")
oncomine[:3]

In [ ]:
onco_df = expand_df(oncomine)
onco_df

In [ ]:
merge_df = nngn_df.merge(onco_df, on=["Gene", "Exon"], how="outer", indicator=True)
merge_df

In [ ]:
nngn_only = merge_df.query('_merge == "left_only"')
nngn_only


In [ ]:
oncomine_only = merge_df.query('_merge == "right_only"')
oncomine_only

In [ ]:
with pd.ExcelWriter("/Users/martinszyska/Dropbox/Icke/Work/Topics/CONAN/LungCancerPanels.xlsx", mode="a") as writer:
    nngn_only.loc[:, ["Gene", "ID_x", "Exon"]].to_excel(writer, sheet_name="nNGM_only", index=False)
    oncomine_only.loc[:, ["Gene", "ID_y", "Exon"]].to_excel(writer, sheet_name="Oncomine_only", index=False)